In [1]:
%%time
#@#main_proc
if True:
    import os
    os.chdir(r"c:\Users\Zver\PycharmProjects\Trade-RL_v2.1\content")
    from os.path import exists
    from scipy.stats import pearsonr
    from calc_error import multi_calc_total_error
    import statsmodels.tools.eval_measures as metrics
    import numpy as np
    from datetime import datetime, timedelta
    import matplotlib.cm as cm
    from scipy.spatial import distance
    import pandas as pd
    from IPython.display import clear_output
    import matplotlib.pyplot as plt
    from numpy import array as ar
    from sklearn.cluster import KMeans
    from sklearn.metrics import mean_squared_error as mse
    from execthatcell import *
    from sklearn.metrics import *
    from sklearn import preprocessing
    from time import sleep
    from tqdm import tqdm
    import multiprocessing
    from threading import Thread
    import copy
    import ftplib
    import ta_py as ta
    from random import randint as rnd
    import requests
    import json

    botToken = '222222222:AAF2f-2bTMstIZFfkAMtOROiHV0qzVbm0xo'
    chatId = 222222222222

    pd.options.mode.chained_assignment = None
    pd.options.display.precision = 10
    pd.set_option('display.max_rows', None)
    plt.style.use('fivethirtyeight')
    plt.rcParams['lines.linewidth'] = 2.0
    plt.rcParams['figure.figsize'] = [9, 5]
    # pd.set_option("precision", 13)


# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#                               MAIN_PROC
# vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv

def norm(data,do=True):
    if do:
        _min = np.min(data)
        _max = np.max(data)
        return (data - _min) / (_max - _min)
    else: return np.array(data)
def GetCurrentTime(DELTA_HOUR=0):
    return (datetime.now()+timedelta(hours=DELTA_HOUR)).strftime("%Y-%m-%d %H:%M:%S")
def calc_candle_sizes(df):
    df["body_size"] = abs(df["open"] - df["close"])
    df["range_size"] = df["high"] - df["low"]
    df["upper_shadow_size"] = abs(df["high"] - df[["close","open"]].max(axis=1))
    df["lower_shadow_size"] = abs(df["low"] - df[["close","open"]].min(axis=1))
    return df

## ------------------------ load db --------------------------------------------
hist_data = pd.read_csv('./root_coins/csv_files/BTCUSDT-15m-data-big.csvx')#('./CNN-TA/root_coins/csv_files/BNM_last_price_db_v5.csv')#
# hist_data = hist_data[['date','open','high','low','close']]

HCL = np.stack([hist_data['high'].to_numpy(),hist_data['close'].to_numpy(),hist_data['low'].to_numpy()],axis=1)
# HL = np.stack([hist_data['high'].to_numpy(),hist_data['low'].to_numpy()],axis=1)

HISTORY_PRICE = ((hist_data['low']+hist_data['high']+hist_data['close'])/3).to_numpy()
hist_len = len(HISTORY_PRICE)

# ---------------------------- add indicators ----------------------------------

# x = ta.ichimoku(HCL)
# align = abs(len(HCL) - len(x))
# ichi = ([[0,0,0,0,0]] * align) + x
# a, b, c, d, e = zip(*ichi)
# ichi_a,ichi_b,ichi_c,ichi_d,ichi_e= np.array(a),\
#                                     np.array(b),\
#                                     np.array(c),\
#                                     np.array(d),\
#                                     np.array(e)


# x = ta.don(HL)
# align = abs(len(HL) - len(x))
# donch = ([[0,0,0]] * align) + x
# a, b, c = zip(*donch)
# donch_a,donch_b,donch_c = np.array(a),\
#                           np.array(b),\
#                           np.array(c)

x = ta.macd(HISTORY_PRICE)
align = abs(hist_len-len(x))
macd = ([0]*align)+x

x = ta.rsi(HISTORY_PRICE)
align = abs(hist_len-len(x))
rsi = ([0]*align)+x

# x = ta.atr(HCL)
# align = abs(hist_len-len(x))
# atr = ([0]*align)+x

# x = ta.asi(HCL)
# align = abs(hist_len-len(x))
# asi = ([0]*align)+x

# x = ta.hull(hist_data['volume'].to_numpy(),21)
# align = abs(hist_len-len(x))
# VOL = ([0]*align)+x

avg_price = HISTORY_PRICE
VOL = hist_data['volume'].to_numpy()
trades = hist_data['trades'].to_numpy()

x = ta.wrsi(HISTORY_PRICE)
align = abs(hist_len-len(x))
wrsi = ([0]*align)+x

x = ta.bandwidth(HISTORY_PRICE)
align = abs(hist_len-len(x))
band = ([0]*align)+x

# ichi_a,ichi_b,ichi_c,ichi_d,ichi_e = hist_data['ichi_a'].to_numpy(),\
#                                      hist_data['ichi_b'].to_numpy(),\
#                                      hist_data['ichi_c'].to_numpy(),\
#                                      hist_data['ichi_d'].to_numpy(),\
#                                      hist_data['ichi_e'].to_numpy()

# donch_a,donch_b,donch_c = hist_data['donch_a'].to_numpy(),\
#                           hist_data['donch_b'].to_numpy(),\
#                           hist_data['donch_c'].to_numpy()

hist_data = calc_candle_sizes(hist_data)
bs,rs,uss,lss = hist_data['body_size'].to_numpy(),\
                hist_data['range_size'].to_numpy(),\
                hist_data['upper_shadow_size'].to_numpy(),\
                hist_data['lower_shadow_size'].to_numpy()

# ---------------------------- end indicators ----------------------------------


## ---------------------- for pattern processing -------------------------------
def __processing(update=False):
    if update: execthatcell("upd_binance_price", cell_marker="#@#", latest=True) #get_binomo_price
    last_data = pd.read_csv('./root_coins/csv_files/BTCUSDT-15m-data.csv')#('./CNN-TA/root_coins/csv_files/BNM_last_price.csv')#
    # last_data = last_data[['date','open','high','low','close']]
    # last_data = pd.concat([history_data, last_data], axis=0).reset_index(drop=True)
    return last_data


def edit_tg_photo(message_id,pic):
    image = pic
    address = f'https://api.telegram.org/bot{botToken}/editMessageMedia'
    media = {"type": "photo", "media": "attach://photo", "caption": GetCurrentTime()}
    data = {"chat_id": chatId, "message_id": message_id, "media": json.dumps(media)}
    with open(image, "rb") as imageFile:
        requests.post(address, files={"photo": imageFile}, data=data)
def send_photo(pic):
    image = pic
    address = f'https://api.telegram.org/bot{botToken}/sendPhoto'
    data = {"chat_id": chatId, "caption": GetCurrentTime()}
    with open(image, "rb") as imageFile:
        result = requests.post(address, files={"photo": imageFile}, data=data).json()
        if result["ok"]:
            return result["result"]["message_id"]
        else:
            raise Exception(result["description"])

path_to_file = 'untitled'
try: os.remove(path_to_file)
except: pass

night_mode = 0
result_table = []
sz_list = [5,15,25,30]#302

# sz_list = [x for x in range(5, 51, 5)]

horizon = 4*12 # 24hour?? :D
TRAIN = 0


while True:
    if exists(path_to_file): break # STOP execution
    night_mode += 1
    for sz in sz_list:

        t_error = 0.0
        n=800
        for _ in range(1):

            if TRAIN:
                n = rnd(100,hist_len-sz-horizon)

                # concat all_in_one
                indexes = range(hist_len) # !!!!!!!!!!! recheck!
                all_stack = np.stack([
                                      # macd,
                                      # rsi,
                                      # atr,
                                      # asi,
                                      avg_price,
                                      # VOL,
                                      # trades,
                                      # wrsi,
                                      # band,
                                      # ichi_a,ichi_b,ichi_c,ichi_d,ichi_e,
                                      # donch_a,donch_b,donch_c,
                                      # bs,rs,uss,lss,
                                      indexes],axis=0)  # last row we save our indexes array

                pattrn = []
                # get pattern seq
                for x in range(len(all_stack)-1):
                    pattrn.append(all_stack[x][n:n+sz].copy())

                print('n=',n)
                _true_price_ = norm(HISTORY_PRICE[n+0:n+sz+horizon].copy())

                # fill 0 ptrn from list
                del_idxs = [i for i in range(n-1,n+sz)]#
                for x in range(len(all_stack)-1):
                    np.put(all_stack[x], del_idxs,[0xBAD])
                    # recheck all sizes for same
                    assert len(all_stack[x]) == (hist_len)

                n += 10
            else:    # ---- TEST ----
                LAST = __processing(update=1)
                PRICE = ((LAST['high']+LAST['low']+LAST['close'])/3).to_numpy() #LAST['close'].to_numpy() #
                # HCL = np.stack([LAST['high'].to_numpy(),
                #                 LAST['close'].to_numpy(),
                #                 LAST['low'].to_numpy()],axis=1)
                # HL  = np.stack([LAST['high'].to_numpy(),
                #                 LAST['low'].to_numpy()],axis=1)
                # last_len = len(PRICE)

                # ---------------- get indicators from last seq ----------------

                L_macd = ta.macd(PRICE)
                L_rsi = ta.rsi(PRICE)
                # L_atr = ta.atr(HCL)
                # L_asi = ta.asi(HCL)
                L_avg_price = PRICE
                L_vol = LAST['volume'].to_numpy()
                L_trades = LAST['trades'].to_numpy()
                L_wrsi = ta.wrsi(PRICE)
                L_band = ta.bandwidth(PRICE)


                # a, b, c, d, e = zip(*ta.ichimoku(HCL))
                # L_ichi_a,L_ichi_b,L_ichi_c,L_ichi_d,L_ichi_e = np.array(a),\
                #                                                np.array(b),\
                #                                                np.array(c),\
                #                                                np.array(d),\
                #                                                np.array(e)

                # a, b, c = zip(*ta.don(HL))
                # L_donch_a,L_donch_b,L_donch_c = np.array(a),\
                #                                 np.array(b),\
                #                                 np.array(c)


                LAST = calc_candle_sizes(LAST)
                L_bs = LAST['body_size'].to_numpy()
                L_rs = LAST['range_size'].to_numpy()
                L_uss = LAST['upper_shadow_size'].to_numpy()
                L_lss = LAST['lower_shadow_size'].to_numpy()


                pattrn = []
                # get pattern seq
                for x in [
                          # L_macd,
                          L_rsi,
                          # L_atr,
                          # L_asi,
                          # L_avg_price,
                          # L_vol,
                          # L_trades,
                          # L_wrsi,
                          # L_band,
                          # L_ichi_a,L_ichi_b,L_ichi_c,L_ichi_d,L_ichi_e,
                          # L_donch_a,L_donch_b,L_donch_c,
                          # L_bs,L_rs,L_uss,L_lss,
                          ]:
                    pattrn.append(x[-sz:].copy())

                _true_price_ = PRICE[-sz:].copy()

                # concat all_in_one
                indexes = range(hist_len) # !!!!!!!!!!! recheck!
                all_stack = np.stack([
                                      # macd,
                                      rsi,
                                      # atr,
                                      # asi,
                                      # avg_price,
                                      # VOL,
                                      # trades,
                                      # wrsi,
                                      # band,
                                      # ichi_a,ichi_b,ichi_c,ichi_d,ichi_e,
                                      # donch_a,donch_b,donch_c,
                                      # bs,rs,uss,lss,
                                      indexes],axis=0)  # last row we save our indexes array

                # --------------------------- end ------------------------------


            # ---------- threading - find similarity -----------------------------
            if True:
                SPLIT = 6
                tasks = range(SPLIT)

                multi_stack = np.array_split(all_stack, SPLIT, axis=1) #hsplit -- split two parts! for threading! =)

                queue = multiprocessing.SimpleQueue()

                for x in tasks:
                    multiprocessing.Process(target=multi_calc_total_error,args=(metrics.rmse,
                                                                                pattrn,
                                                                                multi_stack[x],
                                                                                sz,
                                                                                horizon,
                                                                                1,
                                                                                False,
                                                                                True,
                                                                                queue,)).start()
                error_table = []
                for _ in tasks: error_table += queue.get()
            # ------------------------------------------------------------------

            SLICE = 20
            df = pd.DataFrame(error_table).sort_values('error',ascending=True)[:SLICE]
            debug = df.copy()

            if TRAIN:

                preds_list = []
                __List = []

                for x in range(SLICE):
                    # get top-n index
                    index = int(df.iloc[x]['index'])
                    # get predict from HISTORY_PRICE
                    pred = norm(HISTORY_PRICE[index+0 : index+sz+horizon],True)
                    if (x <= 0): preds_list.append(pred) # add only first 8 element!
                    __List.append({'err':metrics.rmse(_true_price_[:-horizon],pred[:-horizon]),
                                   'vector':pred})


                # df = pd.DataFrame(__List)#.sort_values('err',ascending=True)[:]
                # # print('df_train:',df)
                # for x in range(SLICE):
                #     preds_list.append(df.iloc[x]['vector'])

                # get average from all predict's
                pred_avg = np.mean(np.array(preds_list),axis=0)

                # get error's
                # t_error += distance.euclidean(pct0,pred_avg)

                # Plotting
                plt.title(f'[Train] - {sz}')
                plt.plot(_true_price_,label='actual')
                # for q,x in enumerate(preds_list): plt.scatter(y=x,x=range(len(x)),alpha=0.12)#,label=f'pred{q}')
                plt.plot(pred_avg,label='pred_avg',ls='--',c='red',alpha=0.8)
                # plt.plot([],[],' ',label=f"err: {round(y_pred0['mse'])}")
                plt.axvline(x=len(_true_price_)-horizon-1,c='grey',ls='--')
                plt.legend()
                # plt.savefig('grid.svg', bbox_inches='tight')
                # upload_on_ftp('grid.svg')
                plt.show()
                plt.close()
            else: # ---TEST---it---

                preds_list = []
                tmp_list = []

                for x in range(SLICE):
                    # get top-n index
                    index = int(df.iloc[x]['index'])
                    # get predict from HISTORY_PRICE
                    pred = HISTORY_PRICE[index+0 : index+sz+horizon]
                    if (x <= 0): preds_list.append(norm(pred,False)) # add only first 8 element!
                    tmp_list.append({'err':metrics.rmse(norm(_true_price_,False),norm(pred[:-horizon],False)),
                                   'vector':norm(pred,False)})

                df = pd.DataFrame(tmp_list)
                pred = df.iloc[0]['vector']
                err1 = df.iloc[0]['err']

                df = df.sort_values('err',ascending=True)
                pred_sort = df.iloc[0]['vector']
                err2 = df.iloc[0]['err']


                # get average from all predict's
                pred_avg = np.mean(np.array(preds_list),axis=0)

                # Make plott grid
                rows = 1
                columns = 2
                fig = plt.figure(figsize=(21,6))

                # Plott 1st pic ----------------------
                fig.add_subplot(rows, columns, 1)
                plt.title(f'[Live]- {sz}')
                plt.plot(norm(_true_price_,False),label='actual',linewidth=2.2)
                # plt.plot(norm(pred_avg[:-horizon],True),label='pred_avg',ls='--',c='red',linewidth=2.5,alpha=0.7)
                # plt.axvline(x=len(pred)-horizon-1,c='grey',ls='--')
                plt.legend()

                # Plott 2nd pic ----------------------
                fig.add_subplot(rows, columns, 2)
                # plt.title('[Live]')
                plt.plot(pred_avg[:],label='predict',ls='-',c='red',linewidth=2.2,alpha=0.6)
                plt.plot([],[],' ',label=f"err. {round(err1)}")
                plt.axvline(x=len(pred)-horizon-1,c='grey',ls='--')
                plt.legend()

                # Save plot
                plt.savefig('grid.png', bbox_inches='tight')
                # edit_tg_photo(246874,'grid.png')
                send_photo('grid.png')
                # plt.show()
                plt.close()

        result_table.append({'sz':sz,'t_error':debug.iloc[0]['error']})

    # if TRAIN: break
    # if night_mode>110: break
    # break
    sleep(7*60)

FileNotFoundError: ignored